<a href="https://colab.research.google.com/github/zapakzapak/FaceFusion-Online-Ports/blob/main/DynamiCrafter_colab_576_1024_16bit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import shutil
import re
import sys
import types
import subprocess

# 1. CLEANUP
# Remove existing folder to ensure clean install
if os.path.exists('/content/DynamiCrafter-hf'):
    shutil.rmtree('/content/DynamiCrafter-hf')

%cd /content
# Clone the repository
!GIT_LFS_SKIP_SMUDGE=1 git clone -b dev https://github.com/camenduru/DynamiCrafter-576x1024-hf /content/DynamiCrafter-hf
%cd /content/DynamiCrafter-hf

# Fix for missing prompt file
!mkdir -p prompts/1024
!echo "" > prompts/1024/firework03.png

# 2. INSTALL DEPENDENCIES
# Pin open-clip-torch to 2.20.0 to prevent runtime crash
!pip install -q xformers gradio omegaconf einops pytorch_lightning kornia open-clip-torch==2.20.0 timm av decord huggingface_hub

# 3. DOWNLOAD MODEL (Multi-Mirror Retry)
print("Downloading model checkpoint...")
target_dir = "/content/DynamiCrafter-hf/checkpoints/dynamicrafter_1024_v1"
target_file = "model.ckpt"
target_path = os.path.join(target_dir, target_file)
os.makedirs(target_dir, exist_ok=True)

# List of mirrors to try
mirrors = [
    "https://huggingface.co/Doubiiu/DynamiCrafter/resolve/main/checkpoints/dynamicrafter_1024_v1/model.ckpt",
    "https://huggingface.co/camenduru/DynamiCrafter/resolve/main/dynamicrafter_1024_v1.ckpt",
    "https://huggingface.co/vdo/DynamiCrafter/resolve/main/dynamicrafter_1024_v1.ckpt",
    "https://huggingface.co/checkpoints/DynamiCrafter/resolve/main/model.ckpt"
]

download_success = False
for url in mirrors:
    print(f"Attempting download from: {url}")
    try:
        # Using wget with -O to force filename and -c for resume if possible
        result = subprocess.run(["wget", "-c", "-O", target_path, url], capture_output=True)

        # Verify file size (should be > 1GB)
        if os.path.exists(target_path) and os.path.getsize(target_path) > 1024 * 1024 * 100:
            print("Download successful!")
            download_success = True
            break
        else:
            print("Download failed or file too small. Trying next mirror...")
            if os.path.exists(target_path):
                os.remove(target_path)
    except Exception as e:
        print(f"Error downloading from {url}: {e}")

if not download_success:
    print("CRITICAL ERROR: Could not download model from any mirror. Please check internet connection or HF availability.")
    # We don't exit here to allow debugging, but app will likely fail

# 4. APPLY PATCHES
print("Patching app.py...")
with open('app.py', 'r') as f:
    content = f.read()

# A. Enable Public Share
content = content.replace('share=False', 'share=True')

# B. Fix Black Video (Float32 VAE)
# Define the patch code to inject at the TOP of app.py
global_patch = """
import torch
import types

def patched_vae_decode(self, z, **kwargs):
    # Force VAE decoding to use Float32 and disable autocast to prevent black frames
    with torch.cuda.amp.autocast(enabled=False):
        z = z.to(torch.float32)
        return self._original_decode(z, **kwargs)
"""

# Prepend patch imports
content = global_patch + "\n" + content

# Inject the application logic
# We look for the model.cuda() call
match = re.search(r'^(\s*)model\s*=\s*model\.cuda\(\)', content, re.MULTILINE)

if match:
    indent = match.group(1)
    # Create the indented block using the detected indentation
    patch_logic = f"""{indent}model = model.cuda()
{indent}print("Applying VAE Patch (Float32)...")
{indent}model.first_stage_model.to(torch.float32)
{indent}if not hasattr(model.first_stage_model, '_original_decode'):
{indent}    model.first_stage_model._original_decode = model.first_stage_model.decode
{indent}    model.first_stage_model.decode = types.MethodType(patched_vae_decode, model.first_stage_model)
"""
    content = content.replace(match.group(0), patch_logic)
    print("VAE Patch injected successfully.")
else:
    print("WARNING: Could not find 'model = model.cuda()' to inject patch.")

# Suppress warnings
content = "import warnings\nwarnings.filterwarnings('ignore')\n" + content

with open('app.py', 'w') as f:
    f.write(content)

print("Starting application...")
!python app.py

/content
Cloning into '/content/DynamiCrafter-hf'...
remote: Enumerating objects: 192, done.
remote: Counting objects: 100% (192/192), done.
remote: Compressing objects: 100% (129/129), done.
remote: Total 192 (delta 62), reused 187 (delta 60), pack-reused 0 (from 0)
Receiving objects: 100% (192/192), 8.35 MiB | 18.71 MiB/s, done.
Resolving deltas: 100% (62/62), done.
/content/DynamiCrafter-hf
Attempting download from: https://huggingface.co/Doubiiu/DynamiCrafter/resolve/main/checkpoints/dynamicrafter_1024_v1/model.ckpt
Download failed or file too small. Trying next mirror...
Attempting download from: https://huggingface.co/camenduru/DynamiCrafter/resolve/main/dynamicrafter_1024_v1.ckpt
Download failed or file too small. Trying next mirror...
Attempting download from: https://huggingface.co/vdo/DynamiCrafter/resolve/main/dynamicrafter_1024_v1.ckpt
Download failed or file too small. Trying next mirror...
Attempting download from: https://huggingface.co/checkpoints/DynamiCrafter/resolve/